In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import sys
sys.path.append('/home/mahdi_mousavi/comb-auction')

In [3]:
import torch
import torch.nn as nn
import logging

from comblearn.data import DSFValueFunction
from comblearn.env import CombinatorialAuction

logging.basicConfig(level=20)

In [4]:
import yaml 

with open("config_mod.yaml") as fp:
        cfg = yaml.load(fp, Loader=yaml.FullLoader)

In [5]:
device = "cuda"

def social_welfare(ws, allocation):
    return torch.sum(torch.tensor([w(alloc) for w, alloc in zip(ws, allocation)]).to(device))

In [6]:
auction = CombinatorialAuction(cfg['auction'])

INFO:root:tensor([3., 2., 3., 5., 3., 2., 1., 6.], grad_fn=<SqueezeBackward0>)


INFO:root:tensor([5., 0., 3., 5., 3., 4., 4., 1.], grad_fn=<SqueezeBackward0>)
INFO:root:tensor([6., 0., 5., 3., 0., 4., 0., 2.], grad_fn=<SqueezeBackward0>)


[tensor([0., 1., 0., 1., 1., 0., 0., 1.], device='cuda:0'), tensor([0., 0., 0., 0., 0., 1., 1., 0.], device='cuda:0'), tensor([1., 0., 1., 0., 0., 0., 0., 0.], device='cuda:0')]
Brute Force Search: done.
Optimal Social Welfare: 35.0


In [7]:
allocations, payments = auction.run()

INFO:root:Final allocation calculation...
INFO:root:Bidder ali, loss: 1.502211965931366e-10
INFO:root:Bidder hassan, loss: 1.8568010462516327e-09
INFO:root:Bidder mahdi, loss: 2.8759675387846073e-06
INFO:root:t: 0.2500000000000001/1
INFO:root:t: 0.5000000000000003/1
INFO:root:t: 0.7500000000000006/1
INFO:root:t: 1.0000000000000007/1
INFO:root:Optimal allocation:
INFO:root:(('ali', tensor([0., 1., 0., 0., 0., 0., 0., 1.], device='cuda:0')))
INFO:root:(('hassan', tensor([0., 0., 0., 1., 1., 1., 1., 0.], device='cuda:0')))
INFO:root:(('mahdi', tensor([1., 0., 1., 0., 0., 0., 0., 0.], device='cuda:0')))
INFO:root:Social welfare: 34.999053955078125
INFO:root:Payment calculation..
INFO:root:t: 0.2500000000000001/1
INFO:root:t: 0.5000000000000003/1
INFO:root:t: 0.7500000000000006/1
INFO:root:t: 1.0000000000000007/1
INFO:root:t: 0.2500000000000001/1
INFO:root:t: 0.5000000000000003/1
INFO:root:t: 0.7500000000000006/1
INFO:root:t: 1.0000000000000007/1
INFO:root:t: 0.2500000000000001/1
INFO:root:

In [8]:
auction.bidders[0](torch.tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0'))

tensor([25.], device='cuda:0', grad_fn=<SqueezeBackward4>)

In [9]:
auction.bidders[0](torch.tensor([0., 0., 0., 0., 0., 0., 1., 0.], device='cuda:0')) + auction.bidders[0](torch.tensor([1., 1., 1., 1., 1., 1., 0., 1.], device='cuda:0'))

tensor([25.], device='cuda:0', grad_fn=<AddBackward0>)